In [15]:
import pandas as pd
import tkinter as tk
from tkinter import ttk
import unicodedata
import io
import requests

# Função para determinar o delimitador do arquivo CSV
def determinar_delimitador(content):
    delimiters = [',', ';', '\t']
    for delimiter in delimiters:
        if delimiter in content:
            return delimiter
    return ','  # Delimitador padrão é a vírgula

# Função para remover acentos e caracteres especiais
def remove_accents(text):
    text = ''.join(char for char in unicodedata.normalize('NFD', text) if unicodedata.category(char) != 'Mn')
    return text

# Função para baixar, limpar e filtrar o arquivo CSV
def download_clean_filter(url, selected_columns):
    try:
        # Realiza a solicitação HTTP para baixar o arquivo CSV
        response = requests.get(url)
        response.raise_for_status()

        # Lê o conteúdo do arquivo CSV e determina o delimitador
        content = response.content.decode('utf-8')
        delimiter = determinar_delimitador(content)

        # Lê o arquivo CSV ajustando os parâmetros
        df = pd.read_csv(io.StringIO(content), delimiter=delimiter, quotechar='"', escapechar='\\', na_values=["", "None"])

        # Remove acentos e caracteres especiais dos rótulos das colunas
        df.columns = [remove_accents(col).upper() for col in df.columns]

        # Verifica se as colunas especificadas existem no DataFrame
        selected_columns = [col for col in selected_columns if col in df.columns]

        if not selected_columns:
            selected_columns = df.columns[:4]

        # Aplica os filtros nas colunas selecionadas
        filtered_df = df[selected_columns]

        return filtered_df
    except Exception as e:
        return str(e)  # Retorna a mensagem de erro

# Função para lidar com o botão "Download"
def download_button_clicked():
    url = entry_url.get()
    selected_columns = entry_columns.get().split(',')
    result = download_clean_filter(url, selected_columns)

    if isinstance(result, pd.DataFrame):
        # Limpar a grade existente (se houver)
        for i in result_tree.get_children():
            result_tree.delete(i)

        # Preencher a grade com os dados do DataFrame
        result_tree["columns"] = result.columns
        result_tree.heading("#0", text="Índice")
        result_tree.column("#0", width=50)
        for col in result.columns:
            result_tree.heading(col, text=col)
            result_tree.column(col, width=100)
        for index, row in result.iterrows():
            result_tree.insert("", index, values=[index] + list(row))

        result_label.config(text="Arquivo CSV baixado, limpo e filtrado com sucesso.")
    else:
        result_label.config(text="Ocorreu um erro com arquivo CSV")
        print(result)

# Configuração da janela principal
window = tk.Tk()
window.title("Aplicativo de Leitura, Limpeza e Filtro de Arquivo CSV")

# Rótulo e entrada para a URL do arquivo CSV
url_label = tk.Label(window, text="URL do arquivo CSV:")
url_label.pack()
entry_url = tk.Entry(window, width=50)
entry_url.pack()

# Rótulo e entrada para os IDs das colunas de filtro
columns_label = tk.Label(window, text="IDs das colunas (separados por vírgula):")
columns_label.pack()
entry_columns = tk.Entry(window, width=50)
entry_columns.pack()

# Botão para baixar, limpar e filtrar o arquivo CSV
download_button = tk.Button(window, text="Baixar, Limpar e Filtrar", command=download_button_clicked)
download_button.pack()

# Rótulo para exibir o resultado
result_label = tk.Label(window, text="")
result_label.pack()

# Crie uma árvore (grade) para exibir os dados
result_tree = ttk.Treeview(window)
result_tree.pack()

# Iniciar a interface gráfica
window.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Eduarda\AppData\Local\Temp\ipykernel_19408\2235341220.py", line 67, in download_button_clicked
    result_tree.heading(col, text=col)
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\ttk.py", line 1274, in heading
    return _val_or_dict(self.tk, kw, self._w, 'heading', column)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Users\Eduarda\anaconda3\Lib\tkinter\ttk.py", line 281, in _val_or_dict
    res = tk.call(*(args + options))
          ^^^^^^^^^^^^^^^^^^^^^^^^^^
_tkinter.TclError: Invalid column index COUNTRY (EN)
